In [1]:
import tushare as ts
import tkinter as tk
import datetime
import time
from dateutil.relativedelta import relativedelta
from pandastable import Table
from tkintertable import TableCanvas, TableModel
from tkinter import ttk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import SpectralClustering
from scipy.spatial.distance import cdist
pro = ts.pro_api('f198ddf6f6c8918413793b567bed6269e931f433abd746b26a3c2668')

In [2]:
##思路就是找到每只股票一段时间的涨跌额，有正有负，然后合并其中连续的相同符号的值，举例子就是这样：
##                 股票1=[1,1,1,-1,-1,-1]
##                 合并之后=[3,-3] (因为我觉得既然只是趋势的话连续的涨跌可以直接合并成一个数值)
##然后计算两两股票间的双向Hausdorff距离，得到距离矩阵，将距离矩阵放到谱聚类里面得到结果就行了

In [4]:
##over20%.txt 是我初步选出的股票名单，选用的是roic（投资回报率）作为筛选标准，找出一些大于20%的股票
stocklist=[]
with open('over20%.txt', 'r') as f:
    lines = f.readlines() # 整行读取数据
    for i in lines:
        if i.split()[0][-2:]!='BJ':
            stocklist.append(i.split()[0])
##将txt中的股票读入list中

In [5]:
def OneWayHausdorffDistance(ptSetA, ptSetB):
      # 计算任意向量之间的距离，假设ptSetA有n个向量，ptSetB有m个向量
      # 得到矩阵C（n行m列）Cij代表A中都第i个向量到B中第j向量都距离
    dist= cdist(ptSetA, ptSetB, metric='euclidean')
      # np.min(dist,axis=1):计算每一行的的最小值
      # 即：固定点集A的值，求点集A中到集合B的最小值
    return np.max(np.min(dist, axis=1))
# 计算双向的Hausdorff距离=====>H(ptSetA,ptSetB)=max(h(ptSetA,ptSetB),h(ptSetB,ptSetA))
# ptSetA：输入的第一个点集
# ptSetB：输入的第二个点集
# Hausdorff距离度量了两个点集间的最大不匹配程度
def HausdorffDistance(ptSetA, ptSetB):
  # 计算双向的Hausdorff距离距离
    res = np.array([
        OneWayHausdorffDistance(ptSetA, ptSetB),
        OneWayHausdorffDistance(ptSetB, ptSetA)
  ])
    return np.max(res) 
#HausdorffDistance(B,C)
def DistanceMat(data):
    '''
    功能：计算轨迹段的距离矩阵
    输出：距离矩阵
    '''
    
    
    id_=list(data.keys())
    dist=[]
    for i in id_:
        temp=[]
        for j in id_:
            temp.append(HausdorffDistance(data[i],data[j]))
        dist.append(temp)
    dismat=pd.DataFrame(dist,columns=id_,index=id_)
    return dismat

def integrate(li):
    final=[]
    temp=[]
    temp.append(li[0])
    for i in li[1:]:
        if temp[0]*i>0:
            temp.append(i)
        else:
            final.append(sum(temp))
            temp=[]
            temp.append(i)
    final.append(sum(temp))
    return final

def integrate1(close,change):
        final_close=[]
        final_change=[]
        temp=[]
        temp.append(change[0])
        for i in range(1,len(change)):
            if temp[0]*change[i]>0:
                temp.append(change[i])
            else:
                final_change.append(sum(temp))
                final_close.append(close[i-1])
                temp=[]
                temp.append(change[i])
        final_change.append(sum(temp))
        final_close.append(close[i])
        return final_close,final_change

In [47]:
def baseontrend(k,input_type,n,target):##  k是时间段的数量，input_type是日/月/年，n是分成几类，target是查询人想查询的目标股票，格式为string
    ##输出是包含target在内的list，该list中的所有股票被粗略的分为一类，这一输出应该可以用在下一步里
    stock_list=stocklist
    if target not in stock_list:
        stock_list.append(target)
    
    ydm=datetime.datetime.now().strftime('%Y%m%d')# get the current date
    
    if input_type=='day':
        start_date=(datetime.datetime.now()-datetime.timedelta(days=k)).strftime("%Y%m%d")
    elif input_type=='month':
        start_date=(datetime.datetime.now()-relativedelta(months=k)).strftime("%Y%m%d")
    elif input_type=='year':
        start_date=(datetime.datetime.now()-relativedelta(years=k)).strftime("%Y%m%d")
    else:
        print('The input type is wrong!')
        return
    
    X={}
    #for i in stocklist:
    #    temp=[]
    #    df = pro.daily(ts_code=i,start_date=start_date, end_date=ydm, fields='ts_code,trade_date,change')
    #    change=list(df['change'])
    #    change=integrate(change)
    #    idx=list(range(1,len(change)+1))
    #    temp.append(idx)
    #    temp.append(change)
    #    temp=np.array(temp)
    #    X[i]=pd.DataFrame(temp).T
    for i in stock_list:
        temp=[]
        df = pro.daily(ts_code=i,start_date=start_date, end_date=ydm, fields='ts_code,trade_date,close,change')
        close=list(df['close'])[::-1]
        change=list(df['change'])[::-1]
        close1,change1=integrate1(close,change)
        
        
        idx=list(range(1,len(close1)+1))
        temp.append(idx)
        temp.append(close1)
        temp=np.array(temp)
        X[i]=pd.DataFrame(temp).T
    
    dismat=DistanceMat(X)
    clustering = SpectralClustering(n_clusters=n,assign_labels="discretize",random_state=5,affinity='precomputed').fit(dismat)
    label=clustering.labels_
    
    clusters=[]
    for i in range(n):
        temp1=[]
        for j in range(len(label)):
            if label[j]==i:
                temp1.append(stock_list[j])
        clusters.append(temp1)
    
    cluster_dict={}
    for i in range(n):
        cluster_dict[i]=clusters[i]
    
    same=[]
    for j in range(n):
        if target in cluster_dict[j]:
            same=cluster_dict[j]
            break
            
    return same

In [48]:
start=time.time()
same=baseontrend(2,'month',5,'002242.SZ')
end=time.time()
print(end-start)

13.731619834899902


In [49]:
len(same)

58

In [9]:
def vis(ts_code,kvalue,types):##可视化，这个不重要
        ydm=datetime.datetime.now().strftime('%Y%m%d')
        if types=='day':
            start_date=(datetime.datetime.now()-datetime.timedelta(days=int(kvalue))).strftime("%Y%m%d")
        if types=='month':
            start_date=(datetime.datetime.now()-relativedelta(months=int(kvalue))).strftime("%Y%m%d")
        if types=='year':
            start_date=(datetime.datetime.now()-relativedelta(years=int(kvalue))).strftime("%Y%m%d")
        
        df = pro.daily(ts_code=ts_code, start_date=start_date, end_date=ydm, fields='ts_code,trade_date,close,change')
        l1=[]
        for i in df['close']:
            l1.append(i)
        l1=l1[::-1]
        ltime=[]
        for i in df['trade_date']:
            ltime.append(i)
        ltime=ltime[::-1]
        plt.plot(ltime,l1)
        plt.show()